# 计算不同方法不同类型POI的类型比例值

In [3]:
import arcpy

def calculate_TR(feature_class, methods, poi_types):
    """
    Calculate and update the feature class with new fields representing the density proportions of POI types under each calculation method.
    
    Args:
    - feature_class: Path to the feature class.
    - methods: List of methods used for density calculation.
    - poi_types: List of POI types.
    """
    # Ensure the workspace is set to the directory containing the feature class
    arcpy.env.workspace = r"E:\stu1\Projects\POI2023\POI2023.gdb"
    
    # Prepare the list of original density fields and the new proportion fields
    density_fields = [f"{method}_{poi}" for method in methods for poi in poi_types]
    new_fields = [f"{method}_{poi}_Prop" for method in methods for poi in poi_types]
    
    # Add new proportion fields if they don't exist
    existing_fields = [field.name for field in arcpy.ListFields(feature_class)]
    for field in new_fields:
        if field not in existing_fields:
            arcpy.AddField_management(feature_class, field, "DOUBLE")
            print(f"Added field: {field}")

    # Update cursor to calculate and populate new fields
    all_fields = ['SHAPE@'] + density_fields + new_fields  # 'SHAPE@' ensures geometry is not altered
    with arcpy.da.UpdateCursor(feature_class, all_fields) as cursor:
        for row in cursor:
            
            # Calculate total densities for each method
            total_densities = {}
            for method in methods:
                method_indices = [density_fields.index(f"{method}_{poi}") for poi in poi_types]
                method_values = [row[i+1] for i in method_indices]  # +1 accounts for SHAPE@ geometry field
                total_density = sum(filter(None, method_values))  # Sum, ignoring None values
                total_densities[method] = total_density  # Store total density for each method
                
             # Calculate and set proportion values for each method
            for method in methods:
                for poi in poi_types:
                    density_field = f"{method}_{poi}"
                    proportion_field = f"{method}_{poi}_Prop"
                    # Ensure that density_value is not None
                    density_value = row[density_fields.index(density_field) + 1] if row[density_fields.index(density_field) + 1] is not None else 0
                    # Ensure that total_density is not None and not zero before division
                    total_density = total_densities[method]
                    proportion = (density_value / total_density) if total_density else 0  # Avoid division by zero
                    row[new_fields.index(proportion_field) + 1 + len(density_fields)] = proportion  # +1 for SHAPE@ and offset for density fields

            cursor.updateRow(row)

    print("Updated density proportions.")

         

In [4]:
# Example usage
feature_class = "blocks_initial"
methods = ['FD', 'KD', 'KDB', 'DE_FD', 'DE_KD', 'DE_KDB']
poi_types = ['A', 'B', 'R', 'G', 'S', 'M']
calculate_TR(feature_class, methods, poi_types)

Updated density proportions.


# 处理土地利用类型占比

**获取街区单元内，占比最大的土地利用类型，以及相应的比例值**

具体而言，首先通过一个分类函数来将gridcode的第一位数字转换为相应的土地利用类别名称，接着将此函数应用于计算每个Block_ID的最大土地利用类型和占比

In [ ]:
import arcpy

# 设置工作环境
arcpy.env.workspace = r'E:\stu1\Projects\POI2023\POI2023.gdb'

In [4]:
# 输入街区矢量文件和土地利用汇总表
blocks_layer = 'blocks_initial'
landuse_table = r'E:\stu1\Projects\POI2023\POI2023.gdb\LandUse_Area_Summary'

In [ ]:
# 添加字段
arcpy.AddField_management(blocks_layer, 'MaxLandUseType', 'TEXT')
arcpy.AddField_management(blocks_layer, 'MaxLandUsePct', 'DOUBLE')

In [5]:
# 土地利用类别的字典映射
landuse_type_dict = {
    '1': '耕地',
    '2': '林地',
    '3': '草地',
    '4': '水域',
    '5': '居民点用地',
    '6': '未利用地'
}

In [7]:
# 创建字典来存储每个街区的土地利用类型和相应的总占比
block_landuse_totals = {}

# 使用SearchCursor遍历LandUse_Area_Summary表
with arcpy.da.SearchCursor(landuse_table, ['Block_ID', 'gridcode', 'LandUsePct']) as search_cursor:
    for row in search_cursor:
        block_id = row[0]
        gridcode = str(row[1])
        landuse_pct = row[2]

        # 提取gridcode的第一位作为土地利用类型
        landuse_type_code = gridcode[0]

        # 忽略缺失值
        if landuse_type_code == '0':
            continue

        # 更新字典
        if (block_id, landuse_type_code) not in block_landuse_totals:
            block_landuse_totals[(block_id, landuse_type_code)] = landuse_pct
        else:
            block_landuse_totals[(block_id, landuse_type_code)] += landuse_pct

In [8]:
print(block_landuse_totals)

{(1, '1'): 99.03481832243071, (1, '5'): 0.8373080981903758, (2, '5'): 99.99999983006859, (3, '1'): 39.09645784257672, (3, '5'): 60.85483928497517, (4, '5'): 99.99999991761, (5, '1'): 10.197834193076535, (5, '5'): 89.80216578666398, (6, '1'): 59.63520729390325, (6, '5'): 40.36479721654949, (7, '4'): 75.94072347385887, (7, '5'): 24.059276288960668, (8, '1'): 4.3517798465252655, (8, '4'): 1.8284190864020142, (8, '5'): 93.81980102315622, (9, '1'): 72.07147944154899, (9, '4'): 7.073945916473138, (9, '5'): 20.854574692150155, (10, '5'): 100.0, (11, '4'): 8.452162756076218, (11, '5'): 91.54783720346651, (12, '4'): 93.34717126259578, (12, '5'): 6.652828783033406, (13, '1'): 1.041507039263488, (13, '4'): 98.95849298333151, (14, '1'): 0.0011516581915836839, (14, '4'): 31.627738059900743, (14, '5'): 68.37111030843006, (15, '1'): 13.52232345479253, (15, '4'): 54.58936264482015, (15, '5'): 30.99017997657731, (16, '5'): 100.0, (17, '4'): 83.00372641920677, (17, '5'): 16.996282398434932, (18, '4'): 2

In [9]:
# 创建字典来存储每个街区的最大土地利用类型和百分比
block_max_use = {}

# 遍历block_landuse_totals来确定每个街区的最大土地利用类型和占比
for (block_id, landuse_type_code), total_pct in block_landuse_totals.items():
    if block_id not in block_max_use or total_pct > block_max_use[block_id][1]:
        # 使用字典映射来获取土地利用类型名称
        landuse_type_name = landuse_type_dict.get(landuse_type_code, '未知')
        block_max_use[block_id] = (landuse_type_name, total_pct)

In [10]:
print(block_max_use)

{1: ('耕地', 99.03481832243071), 2: ('居民点用地', 99.99999983006859), 3: ('居民点用地', 60.85483928497517), 4: ('居民点用地', 99.99999991761), 5: ('居民点用地', 89.80216578666398), 6: ('耕地', 59.63520729390325), 7: ('水域', 75.94072347385887), 8: ('居民点用地', 93.81980102315622), 9: ('耕地', 72.07147944154899), 10: ('居民点用地', 100.0), 11: ('居民点用地', 91.54783720346651), 12: ('水域', 93.34717126259578), 13: ('水域', 98.95849298333151), 14: ('居民点用地', 68.37111030843006), 15: ('水域', 54.58936264482015), 16: ('居民点用地', 100.0), 17: ('水域', 83.00372641920677), 18: ('居民点用地', 78.17563756995497), 19: ('居民点用地', 80.89807227930149), 20: ('居民点用地', 80.76314137324684), 21: ('耕地', 64.39270283321709), 22: ('居民点用地', 75.33178008881578), 23: ('居民点用地', 96.59188214344813), 24: ('居民点用地', 100.00000001441654), 25: ('水域', 67.12487400777307), 26: ('居民点用地', 100.0), 27: ('水域', 99.77349868512975), 28: ('居民点用地', 65.75503820979858), 29: ('居民点用地', 65.7338132865811), 30: ('居民点用地', 99.99999999999993), 31: ('居民点用地', 77.7034720399059), 32: ('居民点用地', 78.8083389380

In [11]:
# 使用UpdateCursor更新街区矢量文件
with arcpy.da.UpdateCursor(blocks_layer, ['Block_ID', 'MaxLandUseType', 'MaxLandUsePct']) as update_cursor:
    for row in update_cursor:
        block_id = row[0]

        # 如果这个街区有相应的土地利用记录，则更新
        if block_id in block_max_use:
            # 更新为最大土地利用类型名称和百分比
            row[1] = block_max_use[block_id][0]
            row[2] = block_max_use[block_id][1]

            update_cursor.updateRow(row)

# 识别功能区

对于某一街区，是否有某类功能POI占比超过50%，【是】判断该区域为该功能区，【否】判断是否有某类功能区POI占比超过40%，有一类超过40%，判定为以此功能为主的混合功能区，有两类超过40%，判断该区域为此两类功能的功能混合区，如果都没有超过40%则判定该区域为多功能混合区

In [1]:
import arcpy

def identify_urban_function_with_details(feature_class, method_prefix='FD', thresholds=(0.5, 0.4)):
    """
    Identifies the urban function of each block, dynamically names output fields based on the method prefix,
    adds two new fields to the feature class: one for the urban function category and another for the specific leading categories.
    """
    # Dynamically create field names based on method_prefix
    function_field = f"{method_prefix}_UrbanFuncCat"
    leading_field = f"{method_prefix}_LeadingCat"

    # Ensure the new fields exist, if not, add them
    existing_fields = [f.name for f in arcpy.ListFields(feature_class)]
    if function_field not in existing_fields:
        arcpy.AddField_management(feature_class, function_field, "TEXT", field_length=255)
    if leading_field not in existing_fields:
        arcpy.AddField_management(feature_class, leading_field, "TEXT", field_length=255)

    # Define a dictionary to map field suffixes to abbreviations
    category_abbreviations = {
        'A_Prop': '公服',  # Residential
        'B_Prop': '商业',  # Commercial
        'R_Prop': '居住',  # Recreational
        'G_Prop': '绿地',  # Green Space
        'S_Prop': '交通',  # Services
        'M_Prop': '工业'   # Manufacturing
    }

    # Generate field names for the POI proportions based on the method prefix
    poi_fields = [f'{method_prefix}_{suffix}' for suffix in category_abbreviations.keys()]
    field_to_abbr = {f'{method_prefix}_{key}': val for key, val in category_abbreviations.items()}

    # Update cursor to classify each block and identify leading categories
    with arcpy.da.UpdateCursor(feature_class, poi_fields + [function_field, leading_field]) as cursor:
        for row in cursor:
            proportions = row[:-2]  # Exclude the output fields values
            high_threshold, low_threshold = thresholds
            
            # Identify categories exceeding the high threshold
            high_categories = [field_to_abbr[poi_fields[i]] for i, prop in enumerate(proportions) if prop > high_threshold]
            if high_categories:
                row[-2] = 'Single Dominant'
                row[-1] = '/'.join(high_categories)
            else:
                # Identify categories exceeding the low threshold
                low_categories = [field_to_abbr[poi_fields[i]] for i, prop in enumerate(proportions) if prop > low_threshold]
                if len(low_categories) == 1:
                    row[-2] = 'Single Mixed'
                    row[-1] = low_categories[0]
                elif len(low_categories) >= 2:
                    row[-2] = 'Dual Mixed'
                    row[-1] = '/'.join(low_categories[:2])  # Adjust based on preference
                else:
                    row[-2] = 'Multi Mixed'
                    row[-1] = 'MULTI'

            cursor.updateRow(row)

In [7]:
feature_class = r'E:\stu1\Projects\POI2023\POI2023.gdb\blocks_initial'
method_prefix = 'FD'  
thresholds = (0.5, 0.4)  # Adjust your classification thresholds as needed

identify_urban_function_with_details(feature_class, method_prefix, thresholds)

In [2]:
feature_class = r'E:\stu1\Projects\POI2023\POI2023.gdb\blocks_initial'
method_prefix = 'KD'  
thresholds = (0.5, 0.4)  # Adjust your classification thresholds as needed

identify_urban_function_with_details(feature_class, method_prefix, thresholds)

In [3]:
feature_class = r'E:\stu1\Projects\POI2023\POI2023.gdb\blocks_initial'
method_prefix = 'KDB'  
thresholds = (0.5, 0.4)  # Adjust your classification thresholds as needed

identify_urban_function_with_details(feature_class, method_prefix, thresholds)

In [4]:
feature_class = r'E:\stu1\Projects\POI2023\POI2023.gdb\blocks_initial'
method_prefix = 'DE_FD'  
thresholds = (0.5, 0.4)  # Adjust your classification thresholds as needed

identify_urban_function_with_details(feature_class, method_prefix, thresholds)

In [5]:
feature_class = r'E:\stu1\Projects\POI2023\POI2023.gdb\blocks_initial'
method_prefix = 'DE_KD'  
thresholds = (0.5, 0.4)  # Adjust your classification thresholds as needed

identify_urban_function_with_details(feature_class, method_prefix, thresholds)

In [6]:
feature_class = r'E:\stu1\Projects\POI2023\POI2023.gdb\blocks_initial'
method_prefix = 'DE_KDB'  
thresholds = (0.5, 0.4)  # Adjust your classification thresholds as needed

identify_urban_function_with_details(feature_class, method_prefix, thresholds)

# 计算熵值

In [1]:
import arcpy
import math

def calculate_entropy(feature_class, method_prefix):
    """
    计算指定街区矢量数据集的熵值。
    
    :param feature_class: 街区矢量数据集的路径。
    :param method_prefix: 使用的计算方法前缀，如'FD'。
    :return: 熵值计算结果列表，每个街区一个熵值。
    """
    entropy_results = []
    fields = [f.name for f in arcpy.ListFields(feature_class) if f.name.startswith(method_prefix + '_') and f.name.endswith('_Prop')]
    
    with arcpy.da.SearchCursor(feature_class, fields) as cursor:
        for row in cursor:
            entropy = 0
            for proportion in row:
                if proportion > 0:  # 避免对0取对数
                    entropy -= proportion * math.log(proportion)
            entropy_results.append(entropy)
    
    return entropy_results

In [3]:
# 示例用法
feature_class = r'E:\stu1\Projects\POI2023\POI2023.gdb\blocks_initial'
method_prefix = 'FD'  
entropy_values = calculate_entropy(feature_class, method_prefix)
print(entropy_values)

[0, 1.0809744424453327, 1.0499199323122148, 0.3131310488846401, 0.6874578492430503, 0.5787563565956654, 0, 1.2346160592379039, 1.0652337408147838, 0.9606696407754989, 1.2050620179525326, 1.3442748500226895, 1.3585519639454084, 0.9102005389355095, 0.09391322730246118, 0.40710388292364985, 0, 0.0, 0.9164056067359722, 1.2905952580577644, 0, 0.909060010793549, 1.480341189918912, 1.27551596379219, 1.0690010795091862, 0.20734761731733192, 1.3599683751880811, 1.031787550375439, 1.2985168788443628, 0.3994200804771036, 0.7509428256118771, 0.8552082505181542, 1.0134656453309105, 0.9963089646998984, 0.0, 1.0461393257281968, 1.3008978193750727, 0.6902233800724678, 0.7253862180502647, 1.1926388796279495, 0.8944994948547419, 1.3537189236949236, 0.0, 1.3007299750008123, 1.2172489680023038, 1.192434280768353, 0.0, 1.1188292866376197, 1.333412610759141, 0.8180298019430076, 1.2259711414667664, 0.8661500932219324, 0.0, 1.330899854499546, 1.1798938424193401, 1.0861102350482439, 1.5586059253665878, 1.22106

In [4]:
import arcpy
import math

def calculate_and_store_entropy(feature_class, method_prefix):
    """
    计算指定街区矢量数据集的熵值，并将结果存储到属性表中的新字段。
    
    :param feature_class: 街区矢量数据集的路径。
    :param method_prefix: 使用的计算方法前缀，如'FD'。
    """
    # 构建新字段名称
    entropy_field = f"{method_prefix}_Entropy"
    
    # 检查字段是否存在，如果不存在则创建
    if entropy_field not in [f.name for f in arcpy.ListFields(feature_class)]:
        arcpy.AddField_management(feature_class, entropy_field, "DOUBLE")
    
    # 获取相关字段名称
    fields = [f.name for f in arcpy.ListFields(feature_class) if f.name.startswith(method_prefix + '_') and f.name.endswith('_Prop')]
    fields.append(entropy_field)  # 将熵值字段添加到更新列表中
    
    with arcpy.da.UpdateCursor(feature_class, fields) as cursor:
        for row in cursor:
            entropy = 0
            for proportion in row[:-1]:  # 排除熵值字段
                if proportion > 0:  # 避免对0取对数
                    entropy -= proportion * math.log(proportion)
            row[-1] = entropy  # 更新熵值字段
            cursor.updateRow(row)

In [5]:
# 示例用法
feature_class = r'E:\stu1\Projects\POI2023\POI2023.gdb\blocks_initial'
method_prefix = 'FD'  
calculate_and_store_entropy(feature_class, method_prefix)

In [6]:
method_prefix = 'KD'  
calculate_and_store_entropy(feature_class, method_prefix)

In [7]:
method_prefix = 'KDB'  
calculate_and_store_entropy(feature_class, method_prefix)

In [8]:
method_prefix = 'DE_FD'  
calculate_and_store_entropy(feature_class, method_prefix)

In [9]:
method_prefix = 'DE_KD'  
calculate_and_store_entropy(feature_class, method_prefix)

In [10]:
method_prefix = 'DE_KDB'  
calculate_and_store_entropy(feature_class, method_prefix)

# 结果检验

## 抽取检验集

In [2]:
import arcpy
import random

# 设置环境
arcpy.env.workspace = r"E:\stu1\Projects\POI2023\POI2023.gdb"  
input_feature_class = "blocks_initial"  
output_feature_class = "blocks_accuracy"  # 输出文件名

In [3]:
# 获取输入要素的数量
feature_count = arcpy.GetCount_management(input_feature_class)[0]

# 生成一个随机选取的索引集合
random_indexes = random.sample(range(int(feature_count)), 30)  # 随机选取30个街区

In [5]:
# 选中原矢量文件中的随机街区
# 创建一个临时图层来进行选择
arcpy.MakeFeatureLayer_management(input_feature_class, "temp_layer")

# 构造查询字符串来选择随机索引对应的街区
# 假设有一个名为 "OBJECTID" 的字段用作唯一标识符，这个字段名称可能根据你的数据而不同
oids_selected = ",".join([str(oid) for oid in random_indexes])
query = f"OBJECTID IN ({oids_selected})"

# 使用构造的查询字符串进行选择
arcpy.SelectLayerByAttribute_management("temp_layer", "NEW_SELECTION", query)

# 将选中的街区导出到新的矢量文件
arcpy.CopyFeatures_management("temp_layer", output_feature_class)

print(f"完成随机选取并导出{len(random_indexes)}个街区到'{output_feature_class}'。")

完成随机选取并导出30个街区到'blocks_accuracy'。


In [7]:
import arcpy
import random

# 获取输入要素的数量
feature_count = int(arcpy.GetCount_management(input_feature_class)[0])

# 定义随机选择的次数和每次选择的街区数量
num_selections = 5  # 假设我们要随机选择5次
num_blocks_per_selection = 30  # 每次选择30个街区

# 循环进行随机选择和导出
for i in range(num_selections):
    # 生成随机选取的街区ID集合
    random_indexes = random.sample(range(0, feature_count), num_blocks_per_selection)

    # 构造查询字符串来选择随机索引对应的街区
    # 假设有一个名为 "OBJECTID" 的字段用作唯一标识符，这个字段名称可能根据你的数据而不同
    oids_selected = ",".join([str(oid) for oid in random_indexes])
    query = f"OBJECTID IN ({oids_selected})"

    # 使用构造的查询字符串进行选择
    arcpy.SelectLayerByAttribute_management("temp_layer", "NEW_SELECTION", query)

    # 定义输出文件名
    output_feature_class = f"accuracy_check_set_{i+1}.shp"  # 输出文件名会包含选择的次序

    # 将选中的街区导出到新的矢量文件
    arcpy.CopyFeatures_management("temp_layer", output_feature_class)

    print(f"第{i+1}次随机选取并导出{len(random_indexes)}个街区到'{output_feature_class}'。")



第1次随机选取并导出30个街区到'accuracy_check_set_1.shp'。
第2次随机选取并导出30个街区到'accuracy_check_set_2.shp'。
第3次随机选取并导出30个街区到'accuracy_check_set_3.shp'。
第4次随机选取并导出30个街区到'accuracy_check_set_4.shp'。
第5次随机选取并导出30个街区到'accuracy_check_set_5.shp'。


完成随机选取并导出30个街区到'blocks_accuracy'。


[281, 834, 762, 347, 131, 507, 786, 613, 36, 954, 490, 927, 1012, 665, 771, 855, 1025, 619, 400, 617, 300, 251, 956, 503, 527, 120, 373, 497, 156, 827]
